In [1]:
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, clear_output

import HP_EDA as eda
import HP_Preprocessing as preprocess1
import HP_FeatureEngg as feature_engg
import HP_Accuracy as accuracy
import HP_ExternalFactorInfluence_Accuracy as external_accuracy
import HP_SQL_DBValidation as sql_validation

from HP_EDA import HousePriceEDA
from HP_Preprocessing import HousePricingPreprocessor
from HP_FeatureEngg import *
from HP_Accuracy import ModelPipeline
from HP_ExternalFactorInfluence_Accuracy import *
from HP_SQL_DBValidation import *

# Define login credentials
USERNAME = "lboro"
PASSWORD = "lboro"

# Function to display the login page
def display_login_page():
    clear_output()
    
    # Add an image to the login page
    image_url = r"C:\Users\Keerthy M Ganesh\OneDrive - Loughborough University\Project_HousePricePrediction\Datasource"  # Replace with the URL of your image
    login_image = widgets.Image(
        value=open("login_image.jpeg", "rb").read(),  # Load the image from a file
        format='jpeg',
        layout=Layout(height='200px', width='350px')
    )

    # Username and password input fields with icons
    username = widgets.Text(
        placeholder='Enter your username',
        layout=Layout(width='300px')
    )
    username_icon = widgets.HTML(
        value="<i class='fa fa-user' style='font-size:24px;'></i>",
        layout=Layout(width='30px')
    )
    username_box = widgets.HBox([username_icon, username])

    password = widgets.Password(
        placeholder='Enter your password',
        layout=Layout(width='300px')
    )
    password_icon = widgets.HTML(
        value="<i class='fa fa-lock' style='font-size:24px;'></i>",
        layout=Layout(width='30px')
    )
    password_box = widgets.HBox([password_icon, password])

    # Login button
    login_button = widgets.Button(
        description='Login',
        button_style='primary',
        layout=Layout(width='330px')
    )
    login_output = widgets.Output()
    
    def handle_login(btn):
        if username.value == USERNAME and password.value == PASSWORD:
            login_output.clear_output()
            display_main_application()
        else:
            login_output.clear_output()
            with login_output:
                display(widgets.HTML("<p style='color: red;'>Invalid username or password</p>"))
    
    login_button.on_click(handle_login)
    
    display(widgets.VBox([login_image, username_box, password_box, login_button, login_output]))

# Function to display the main application page
def display_main_application():
    clear_output()

    # Define the function to handle changes
    def display_instructions(change):
        with output:
            clear_output()
            if change['new'] == 'Yes':
                display(instructions_html)

    # Create a styled HTML widget for instructions
    instructions_html = widgets.HTML(
        value="<div style='background-color: #f0f8ff; border: 2px solid #3498db; padding: 15px; border-radius: 5px;'>"
              "<h2 style='color: #3498db;'>Instructions for Using the Interface</h2>"
              "<ol>"
              "<B>Note: Only when the first three buttons are clicked, the next set of three buttons would be enabled</B>"
              "<li>First, click the 'EDA' button to perform Exploratory Data Analysis.</li>"
              "<li>Next, click the 'Preprocessing' button to preprocess the data.</li>"
              "<li>Then, click the 'Feature Engineering' button to perform feature engineering tasks.</li>"
              "<li>After that, click the 'Accuracy' button to train and evaluate the model.</li>"
              "<li>Next, click the 'External Factor Accuracy' button to evaluate the influence of external factors.</li>"
              "<li>Finally, click the 'SQL DB Validation' button to validate data between SQL database and local files.</li>"
              "</ol>"
              "</div>"
    )

    # Create radio buttons for the instructions prompt
    instructions_radio = widgets.RadioButtons(
        options=['Yes', 'No'],
        layout=widgets.Layout(display='flex', flex_flow='row', justify_content='space-between')
    )
    instructions_radio.observe(display_instructions, names='value')

    # Create a label for the prompt
    instructions_label = widgets.Label("Do you want to read the Instructions?")

    # Create an HBox to display the radio buttons horizontally
    radio_hbox = widgets.HBox([instructions_radio])

    # Create a VBox to display the label on top and radio buttons below
    vbox = widgets.VBox([instructions_label, radio_hbox])

    # Create an Output widget to manage the display output
    output = widgets.Output()

    # Display the VBox and the Output widget
    display(vbox)
    display(output)

    #####################################
    ########## Event Handlers ###########
    #####################################

    # Button Press States
    button_states = {
        'EDA': False,
        'Preprocessing': False,
        'Feature Engineering': False
    }

    # Define the buttons to be enabled/disabled
    second_buttons = []

    def update_button_states():
        """
        Update the states of the second set of buttons based on the state of the first set.
        """
        all_first_buttons_pressed = all(button_states.values())
        state = 'normal' if all_first_buttons_pressed else 'disabled'

        for button in second_buttons:
            button.disabled = not all_first_buttons_pressed
            button.style.button_color = '#D3D3D3' if not all_first_buttons_pressed else '#007bff'  # Grey for disabled, Blue for enabled

    # Event handlers for the buttons
    out_eda = widgets.Output()
    @out_eda.capture()
    def clicked_btn_eda(btn):
        """
        Handler function for the EDA button.
        """
        out_eda.clear_output()
        csv_path = "housing_london_dataset.csv"
        eda = HousePriceEDA(csv_path)
        eda.perform_eda()
        button_states['EDA'] = True
        update_button_states()

    out_preprocessing = widgets.Output()
    @out_preprocessing.capture()
    def clicked_btn_preprocessing(btn):
        """
        Handler function for the preprocessing button.
        """
        out_preprocessing.clear_output()
        house_pricing_file_name = 'housing_london_dataset.csv'
        lookup_file_name = 'lkp_pc_loc.csv'
        output_file_name = 'temp_housing_london_loc.csv'

        preprocessor = HousePricingPreprocessor(house_pricing_file_name, lookup_file_name)
        preprocessor.preprocess_house_pricing_data()
        preprocessor.handle_missing_values()
        preprocessor.handle_duplicates()
        preprocessor.one_hot_encode('house_type')
        preprocessor.remove_one_hot_columns('house_type')  # Remove one-hot columns
        preprocessor.remove_outliers('price')
        preprocessor.normalize_column('price')
        preprocessor.denormalize_column('price')

        preprocessor.export_to_csv(output_file_name)

        # Display records after all processing
        preprocessor.display_records()

        button_states['Preprocessing'] = True
        update_button_states()

    out_feature_engg = widgets.Output()
    @out_feature_engg.capture()
    def clicked_btn_feature_engg(btn):
        """
        Handler function for the feature engineering button.
        """
        out_feature_engg.clear_output()
        FeatEngg = FeatureEngg()

        # Load and geocode data
        input_file_name = 'lkp_pc_nearest_station.csv'
        output_file_name = 'temp_pc_ns_coordinates.csv'
        FeatEngg.load_and_geocode_data(input_file_name, output_file_name)

        # Merge and calculate distance
        housing_file_name = 'temp_housing_london_loc.csv'
        coordinates_file_name = 'temp_pc_ns_coordinates.csv'
        output_file_merged_name = 'london_housing_with_stations.csv'
        FeatEngg.merge_and_calculate_distance(housing_file_name, coordinates_file_name, output_file_merged_name)

        # Add crime rate factor
        housing_with_stations_file_name = 'london_housing_with_stations.csv'
        crime_file_name = 'lkp_crimerate_influence.csv'
        final_output_file_name = 'london_housing_with_stations_crimerate.csv'
        FeatEngg.add_crime_rate_factor(housing_with_stations_file_name, crime_file_name, final_output_file_name)

        button_states['Feature Engineering'] = True
        update_button_states()

    def create_accuracy_checkboxes():
        """
        Creates and returns a VBox of checkboxes for different accuracy models.
        """
        checkbox_options = {
            'Random Forest': widgets.Checkbox(value=False, description='Random Forest'),
            'Linear Regression': widgets.Checkbox(value=False, description='Linear Regression'),
            'XGBoost': widgets.Checkbox(value=False, description='XGBoost'),
            'SVM': widgets.Checkbox(value=False, description='SVM'),
            'Custom Loss Gradient Boosting': widgets.Checkbox(value=False, description='Custom Loss Gradient Boosting'),
            'Stacking Regressor': widgets.Checkbox(value=False, description='Stacking Regressor'),
            'All': widgets.Checkbox(value=False, description='All'),
            'Predictions': widgets.Checkbox(value=False, description='Predictions')
        }

        vbox = widgets.VBox(list(checkbox_options.values()))

        return vbox, checkbox_options

    out_accuracy = widgets.Output()
    @out_accuracy.capture()
    def clicked_btn_accuracy(btn):
        """
        Handler function for the accuracy button.
        """
        out_accuracy.clear_output()

        # Create and display the checkboxes
        accuracy_vbox, checkbox_options = create_accuracy_checkboxes()
        display(accuracy_vbox)

        def process_selected_models(button):
            selected_models = [model for model, checkbox in checkbox_options.items() if checkbox.value]
            print("Selected models:", selected_models)  # For debugging purposes

            if 'Linear Regression' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_linear_regression()
                pipeline.display_results()
                #pipeline.plot_model_performance()
            if 'XGBoost' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_xgboost()
                pipeline.display_results()
                #pipeline.plot_model_performance()
            if 'Random Forest' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_random_forest()
                pipeline.display_results()
                #pipeline.plot_model_performance()
            if 'SVM' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_svm()
                pipeline.display_results()
                #pipeline.plot_model_performance()
            if 'Custom Loss Gradient Boosting' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_custom_loss_gradient_boosting()
                pipeline.display_results()
                #pipeline.plot_model_performance()
            if 'Stacking Regressor' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_stacking_regressor()
                pipeline.plot_model_performance()
                pipeline.plot_mape_performance()  
                pipeline.display_results()
            if 'Predictions' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.predicted_price()
                pipeline.save_predictions_to_csv('london_housing_with_predictions.csv')
                pipeline.train_xgboost()
                pipeline.plot_price_vs_predicted()
                pipeline.plot_residuals()
                pipeline.plot_actual_vs_predicted_lines()
                pipeline.plot_qq_plot()
            if 'All' in selected_models:
                file_name = "london_housing_with_stations.csv"
                pipeline = ModelPipeline(file_name)
                pipeline.load_data()
                pipeline.preprocess_data()
                pipeline.train_linear_regression()
                pipeline.train_xgboost()
                pipeline.train_random_forest()
                pipeline.train_svm()
                pipeline.train_custom_loss_gradient_boosting()
                pipeline.train_stacking_regressor()
                pipeline.plot_model_performance()
                pipeline.plot_mape_performance()  
                pipeline.display_results()



        # Create and display the "Submit" button
        submit_button = widgets.Button(description="Submit", button_style='success')
        display(submit_button)
        submit_button.on_click(process_selected_models)

    out_external_factor_accuracy = widgets.Output()
    def run_external_factor_accuracy(file_path):
        with out_external_factor_accuracy:
             ExternalFactorInfAccuracy.run(file_path)

    # Capture the output in the widget
    @out_external_factor_accuracy.capture()
    def on_external_factor_dropdown_change(change):
        if change['new'] == 'Linear Regression':
            file_path = r"london_housing_with_stations_crimerate.csv"
            predictor = ExternalFactorInfAccuracy.run(file_path)


    # Handler function for the external factor accuracy button
    @out_external_factor_accuracy.capture(clear_output=True)
    def clicked_btn_external_factor_accuracy(btn):
        external_factor_dropdown = widgets.Dropdown(
            options=['Select a method', 'Linear Regression'],
            value='Select a method',
            description='Method:',
            disabled=False
        )
        external_factor_dropdown.observe(on_external_factor_dropdown_change, names='value')
        display(external_factor_dropdown)

    out_sql_db_validation = widgets.Output()
    @out_sql_db_validation.capture()
    def clicked_btn_sql_db_validation(btn):
        """
        Handler function for the SQL DB validation button.
        """
        out_sql_db_validation.clear_output()
        lookup_file_name = "lkp_pc_loc.csv"
        housing_file_name = "housing_london_dataset.csv"
        output_file_name = "temp_housing_london_loc.csv"

        processor = SqlDbValidation(lookup_file_name, housing_file_name, output_file_name)
        processor.process()

    #####################################
    ###### Additional GUI Elements ######
    #####################################

    # Define the buttons
    eda_button = widgets.Button(description='EDA', icon='eye', layout=Layout(width='auto', height='auto'))
    preprocessing_button = widgets.Button(description='Preprocessing', icon='cogs', layout=Layout(width='auto', height='auto'))
    feature_engg_button = widgets.Button(description='Feature Engineering', icon='flask', layout=Layout(width='auto', height='auto'))

    accuracy_button = widgets.Button(description='Accuracy', icon='bar-chart', layout=Layout(width='auto', height='auto'), disabled=True, style={'button_color': '#D3D3D3'})
    external_factor_accuracy_button = widgets.Button(description='External Factor Accuracy', icon='balance-scale', layout=Layout(width='auto', height='auto'), disabled=True, style={'button_color': '#D3D3D3'})
    sql_db_validation_button = widgets.Button(description='SQL DB Validation', icon='database', layout=Layout(width='auto', height='auto'), disabled=True, style={'button_color': '#D3D3D3'})

    # Assign event handlers
    eda_button.on_click(clicked_btn_eda)
    preprocessing_button.on_click(clicked_btn_preprocessing)
    feature_engg_button.on_click(clicked_btn_feature_engg)
    accuracy_button.on_click(clicked_btn_accuracy)
    external_factor_accuracy_button.on_click(clicked_btn_external_factor_accuracy)
    sql_db_validation_button.on_click(clicked_btn_sql_db_validation)

    # Add the second buttons to the list
    second_buttons = [accuracy_button, external_factor_accuracy_button, sql_db_validation_button]

    # Layout
    tab_eda = widgets.VBox([eda_button, out_eda])
    tab_preprocessing = widgets.VBox([preprocessing_button, out_preprocessing])
    tab_feature_engg = widgets.VBox([feature_engg_button, out_feature_engg])
    tab_accuracy = widgets.VBox([accuracy_button, out_accuracy])
    tab_external_factor_accuracy = widgets.VBox([external_factor_accuracy_button, out_external_factor_accuracy])
    tab_sql_db_validation = widgets.VBox([sql_db_validation_button, out_sql_db_validation])

    tabs = widgets.Tab(children=[tab_eda, tab_preprocessing, tab_feature_engg, tab_accuracy, tab_external_factor_accuracy, tab_sql_db_validation])

    # Set the titles of the tabs
    tabs.set_title(0, 'EDA')
    tabs.set_title(1, 'Preprocessing')
    tabs.set_title(2, 'Feature Engineering')
    tabs.set_title(3, 'Accuracy')
    tabs.set_title(4, 'External Factor Accuracy')
    tabs.set_title(5, 'SQL DB Validation')

    # Create the header using an HTML widget
    heading_style = (
        'background-color: #3498db; color: white; font-size: 24px; '
        'padding: 10px; text-align: center; width: 100%;'
    )
    header_html = widgets.HTML("<div style='" + heading_style + "'>HOUSE PRICE PREDICTION</div>")

    # Display the layout
    display(widgets.VBox([header_html, tabs]))

# Display the login page initially
display_login_page()
